In [1]:
# Import dependencies
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import timeit

from datetime import datetime
from config import weather_api_key

In [2]:
file_path="Resources/Electric_Vehicle_Charging_Stations_in_New_York.csv"
Charger_df=pd.read_csv(file_path)
Charger_df

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,Latitude,Longitude,Date Last Confirmed,ID,Updated At,Owner Type Code,Federal Agency ID,Federal Agency Name,Open Date,EV Connector Types
0,ELEC,Town of Penfield - Library,1985 Baird Rd,NaN,Penfield,NY,14526,NaN,585-340-8720,E,...,43.137476,-77.465214,06/07/2021,49941,2022-02-10 19:42:29,LG,NaN,NaN,04/01/2012,J1772
1,ELEC,Village of Webster,28 West Main Street,NaN,Webster,NY,14580,NaN,866-816-7584,E,...,43.211386,-77.431663,11/10/2022,116756,2022-11-10 02:00:34,NaN,NaN,NaN,08/31/2018,J1772
2,ELEC,HOTSARA CHARGER STATION 1,100 Main St,NaN,Saranac Lake,NY,12983,NaN,888-758-4389,E,...,44.326862,-74.130150,11/10/2022,149787,2022-11-10 00:57:46,NaN,NaN,NaN,12/07/2019,J1772
3,ELEC,Houghton College,9736 Rt 19,NaN,Houghton,NY,14744,NaN,866-816-7584,E,...,42.423869,-78.154428,11/10/2022,157942,2022-11-10 02:00:45,NaN,NaN,NaN,05/15/2020,J1772
4,ELEC,NYC FLEET DPRMBLOT8_1_L3,920 Father Capodanno Blvd,NaN,Midland Beach,NY,10305,NaN,888-758-4389,E,...,40.572636,-74.086140,11/10/2022,182295,2022-11-10 01:05:12,NaN,NaN,NaN,01/27/2021,CHADEMO J1772COMBO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,ELEC,W DELAVAN DELAVAN (1),722 W Delavan Ave,NaN,Buffalo,NY,14222,NaN,888-758-4389,E,...,42.922743,-78.873098,11/10/2022,154370,2022-11-10 01:01:48,NaN,NaN,NaN,03/07/2020,J1772
3080,ELEC,HAMPTON INN STATION 1,1120 South Ave,NaN,Staten Island,NY,10314,NaN,888-758-4389,E,...,40.612696,-74.179012,11/10/2022,168146,2022-11-10 01:05:20,NaN,NaN,NaN,11/05/2020,J1772
3081,ELEC,SRYMCASSB BATTENKILL 1.1,15 Van Ness Ave,NaN,Greenwich,NY,12834,NaN,888-758-4389,E,...,43.093964,-73.506775,11/10/2022,183460,2022-11-10 01:06:06,NaN,NaN,NaN,01/27/2021,J1772
3082,ELEC,PAUL MILLER VW VW_408107_A,5865 Rome-Taberg Rd,NaN,Rome,NY,13440,NaN,888-758-4389,E,...,43.233239,-75.497597,11/10/2022,152393,2022-11-10 00:42:37,NaN,NaN,NaN,01/11/2020,J1772


In [3]:
Charger_df.columns


Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
       'EV Other Info', 'EV Network', 'EV Network Web', 'Geocode Status',
       'Latitude', 'Longitude', 'Date Last Confirmed', 'ID', 'Updated At',
       'Owner Type Code', 'Federal Agency ID', 'Federal Agency Name',
       'Open Date', 'EV Connector Types'],
      dtype='object')

In [4]:
Charger_df = Charger_df [['ID','Station Name', 'City', 'ZIP']]
Charger_df

,ID,Station Name,City,ZIP
0,49941,Town of Penfield - Library,Penfield,14526
1,116756,Village of Webster,Webster,14580
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983
3,157942,Houghton College,Houghton,14744
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305
...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222
3080,168146,HAMPTON INN STATION 1,Staten Island,10314
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440


In [5]:
Charger_df['City'].nunique()

570

In [6]:
from uszipcode import SearchEngine

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
#Creating a dictionary that shows which county each zipcode is in
zipcodes_to_search = list(Charger_df['ZIP'].unique())
search = SearchEngine()
zip_map = {}

# excluding the zipcode not corresponded to any county
for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        zip_info = search.by_zipcode(zipcode).county
        zip_map[zipcode] = zip_info

zip_map

{14526: 'Monroe County',
 14580: 'Monroe County',
 12983: 'Franklin County',
 14744: 'Allegany County',
 10305: 'Richmond County',
 14227: 'Erie County',
 11220: 'Kings County',
 13090: 'Onondaga County',
 10954: 'Rockland County',
 14853: 'Tompkins County',
 11040: 'Nassau County',
 12205: 'Albany County',
 11976: 'Suffolk County',
 12590: 'Dutchess County',
 11212: 'Kings County',
 12534: 'Columbia County',
 14204: 'Erie County',
 10901: 'Rockland County',
 12180: 'Rensselaer County',
 10065: 'New York County',
 13027: 'Onondaga County',
 14621: 'Monroe County',
 11954: 'Suffolk County',
 11710: 'Nassau County',
 12110: 'Albany County',
 11901: 'Suffolk County',
 12020: 'Saratoga County',
 12477: 'Ulster County',
 13441: 'Oneida County',
 12144: 'Rensselaer County',
 14223: 'Erie County',
 14850: 'Tompkins County',
 14616: 'Monroe County',
 12054: 'Albany County',
 12866: 'Saratoga County',
 14731: 'Cattaraugus County',
 10022: 'New York County',
 14072: 'Erie County',
 12065: 'Sarat

In [8]:
#mapping county information for each charger based on zipcode
Charger_df['County'] = Charger_df['ZIP'].map(zip_map) 
Charger_df.head()

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,Station Name,City,ZIP,County
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County
1,116756,Village of Webster,Webster,14580,Monroe County
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County
3,157942,Houghton College,Houghton,14744,Allegany County
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County


In [9]:
import pgeocode
zipcodes_to_search = list(Charger_df['ZIP'].unique())

nomi = pgeocode.Nominatim('us')


latitude_map = {}

for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        latitude_info = nomi.query_postal_code(str(zipcode)).latitude
        latitude_map[zipcode] = latitude_info

latitude_map

Charger_df['latitude'] = Charger_df['ZIP'].map(latitude_map) 
Charger_df

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Station Name,City,ZIP,County,latitude
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County,43.1396
1,116756,Village of Webster,Webster,14580,Monroe County,43.2196
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County,44.3243
3,157942,Houghton College,Houghton,14744,Allegany County,42.4228
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County,40.5973
...,...,...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222,Erie County,42.9164
3080,168146,HAMPTON INN STATION 1,Staten Island,10314,Richmond County,40.6039
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834,Washington County,43.0947
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440,Oneida County,43.2193


In [10]:
zipcodes_to_search = list(Charger_df['ZIP'].unique())

nomi = pgeocode.Nominatim('us')

longitude_map = {}

for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        longitude_info = nomi.query_postal_code(str(zipcode)).longitude
        longitude_map[zipcode] = longitude_info

longitude_map

Charger_df['longitude'] = Charger_df['ZIP'].map(longitude_map) 
Charger_df

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Station Name,City,ZIP,County,latitude,longitude
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County,43.1396,-77.4560
1,116756,Village of Webster,Webster,14580,Monroe County,43.2196,-77.4616
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County,44.3243,-74.1330
3,157942,Houghton College,Houghton,14744,Allegany County,42.4228,-78.2063
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County,40.5973,-74.0768
...,...,...,...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222,Erie County,42.9164,-78.8763
3080,168146,HAMPTON INN STATION 1,Staten Island,10314,Richmond County,40.6039,-74.1472
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834,Washington County,43.0947,-73.5030
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440,Oneida County,43.2193,-75.4498


In [15]:
Charger_df['median_home_value'] = Charger_df['ZIP'].map(median_home_value_map) 
Charger_df

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Station Name,City,ZIP,County,latitude,longitude,population_density,median_home_value,occupied_housing_units
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County,43.1396,-77.4560,1184.0,169000.0,8012.0
1,116756,Village of Webster,Webster,14580,Monroe County,43.2196,-77.4616,1188.0,179100.0,20116.0
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County,44.3243,-74.1330,36.0,162400.0,3524.0
3,157942,Houghton College,Houghton,14744,Allegany County,42.4228,-78.2063,120.0,105500.0,378.0
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County,40.5973,-74.0768,9819.0,415300.0,15145.0
...,...,...,...,...,...,...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222,Erie County,42.9164,-78.8763,10937.0,228700.0,7074.0
3080,168146,HAMPTON INN STATION 1,Staten Island,10314,Richmond County,40.6039,-74.1472,6235.0,437500.0,30709.0
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834,Washington County,43.0947,-73.5030,76.0,179500.0,2634.0
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440,Oneida County,43.2193,-75.4498,381.0,97000.0,17161.0


In [16]:
#Creating a dictionary that shows which latitude each zipcode is in
zipcodes_to_search = list(Charger_df['ZIP'].unique())
search = SearchEngine()
median_household_income_map = {}

# excluding the zipcode not corresponded to any county
for zipcode in zipcodes_to_search:
    if zipcode == 12081:
        pass
    elif zipcode == 12217:
        pass
    else:
        median_household_income_info = search.by_zipcode(zipcode).median_household_income
        median_household_income_map[zipcode] = median_household_income_info

median_home_value_map

Charger_df['median_household_income'] = Charger_df['ZIP'].map(median_household_income_map) 
Charger_df

/Users/hosseindabiri/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Station Name,City,ZIP,County,latitude,longitude,population_density,median_home_value,occupied_housing_units,median_household_income
0,49941,Town of Penfield - Library,Penfield,14526,Monroe County,43.1396,-77.4560,1184.0,169000.0,8012.0,77386.0
1,116756,Village of Webster,Webster,14580,Monroe County,43.2196,-77.4616,1188.0,179100.0,20116.0,69189.0
2,149787,HOTSARA CHARGER STATION 1,Saranac Lake,12983,Franklin County,44.3243,-74.1330,36.0,162400.0,3524.0,47329.0
3,157942,Houghton College,Houghton,14744,Allegany County,42.4228,-78.2063,120.0,105500.0,378.0,50298.0
4,182295,NYC FLEET DPRMBLOT8_1_L3,Midland Beach,10305,Richmond County,40.5973,-74.0768,9819.0,415300.0,15145.0,70758.0
...,...,...,...,...,...,...,...,...,...,...,...
3079,154370,W DELAVAN DELAVAN (1),Buffalo,14222,Erie County,42.9164,-78.8763,10937.0,228700.0,7074.0,52073.0
3080,168146,HAMPTON INN STATION 1,Staten Island,10314,Richmond County,40.6039,-74.1472,6235.0,437500.0,30709.0,77242.0
3081,183460,SRYMCASSB BATTENKILL 1.1,Greenwich,12834,Washington County,43.0947,-73.5030,76.0,179500.0,2634.0,60913.0
3082,152393,PAUL MILLER VW VW_408107_A,Rome,13440,Oneida County,43.2193,-75.4498,381.0,97000.0,17161.0,49755.0


In [17]:
# Exporting the DataFrame as WeatherPy_Database.csv into the Weather_Database folder
# defining the path
output_data_path = "Resources/Demographic_Database.csv"

# exporting data into csv file
Charger_df.to_csv(output_data_path, index_label="city_id")

In [19]:
lats = list(Charger_df['latitude'])
lngs = list(Charger_df['longitude'])

# pack the random coordinates into a zip
coordinates_list = zip(lats,lngs)

coordinates_list

In [20]:
coordinates= list(coordinates_list)
print(coordinates)

[(43.1396, -77.456), (43.2196, -77.4616), (44.3243, -74.133), (42.4228, -78.2063), (40.5973, -74.0768), (42.8853, -78.7462), (40.6412, -74.0133), (43.1528, -76.2235), (41.0977, -74.0109), (42.4474, -76.4837), (40.7294, -73.6828), (42.7198, -73.8207), (40.9209, -72.3491), (41.595, -73.8876), (40.6625, -73.9145), (42.247, -73.7552), (42.884, -78.8597), (41.1177, -74.1241), (42.7287, -73.6683), (40.7651, -73.9638), (43.162, -76.3237), (43.1834, -77.6043), (41.0459, -71.944), (42.247, -73.7552), (40.6729, -73.5365), (42.7462, -73.763), (40.9262, -72.652), (43.005, -73.8486), (42.0738, -73.9797), (43.2264, -75.4083), (42.6359, -73.7219), (42.9731, -78.845), (44.3243, -74.133), (42.4406, -76.4966), (43.2346, -77.6577), (42.6158, -73.8373), (42.4228, -78.2063), (43.0708, -73.7408), (42.2959, -78.6606), (43.0708, -73.7408), (40.7571, -73.9657), (43.0183, -78.9591), (42.8499, -73.7851), (40.9271, -73.0127), (42.7005, -74.9243), (41.7464, -74.1092), (40.7444, -73.0542), (42.7003, -73.8575), (40.

In [21]:
# Get the nearest city using the citipy module
# Create a list to hold the nearest cities
nearest_cities=[]

# detrmining the nearest city for each pair of lats and lngs
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0], coordinate[1]).city_name
#including unique city in the list
    if city not in nearest_cities:
        nearest_cities.append(city)
#printing the city count to see how many cities we have in the list

len(nearest_cities)

170

In [22]:
#Perform an API call with the OpenWeatherMap.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [23]:
import time
from datetime import datetime
# Create an empty list to hold the weather data.
city_data = []
# indicating the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(nearest_cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(55)
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_info = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_info["coord"]["lat"]
        city_lng = city_info["coord"]["lon"]
        city_country = city_info["sys"]["country"]
        city_max_temp = city_info["main"]["temp_max"]
        city_min_temp = city_info["main"]["temp_min"]
        city_humidity = city_info["main"]["humidity"]
        city_clouds = city_info["clouds"]["all"]
        city_wind = city_info["wind"]["speed"]
        description = city_info['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_info["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Min Temp": city_min_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | brighton
Processing Record 2 of Set 1 | huntingdon
Processing Record 3 of Set 1 | olean
Processing Record 4 of Set 1 | bayonne
Processing Record 5 of Set 1 | cheektowaga
Processing Record 6 of Set 1 | new york
Processing Record 7 of Set 1 | fairmount
Processing Record 8 of Set 1 | nanuet
Processing Record 9 of Set 1 | ithaca
Processing Record 10 of Set 1 | new hyde park
Processing Record 11 of Set 1 | albany
Processing Record 12 of Set 1 | old saybrook
Processing Record 13 of Set 1 | arlington
Processing Record 14 of Set 1 | kingston
Processing Record 15 of Set 1 | buffalo
Processing Record 16 of Set 1 | suffern
Processing Record 17 of Set 1 | troy
Processing Record 18 of Set 1 | guttenberg
Processing Record 19 of Set 1 | rochester
Processing Record 20 of Set 1 | groton
Processing Record 21 of Set 1 | bellmore
Processing Record 22 of Set 1 | watervliet
Processing Record 23 of Set 1 | mastic
Proce

In [24]:
len (city_data)

170

In [26]:
# adding  the weather data to a new DataFrame
city_data_df=pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Brighton,GB,50.8284,-0.1395,48.79,44.10,82,7,7.00,clear sky
1,Huntingdon,GB,52.3305,-0.1865,44.80,41.86,84,100,12.12,overcast clouds
2,Olean,US,42.0776,-78.4297,24.96,21.54,79,100,8.05,light snow
3,Bayonne,FR,43.3333,-1.4167,55.13,52.75,94,75,4.61,broken clouds
4,Cheektowaga,US,42.9034,-78.7548,27.16,23.94,56,20,20.71,few clouds


In [27]:
# Exporting the DataFrame as WeatherPy_Database.csv into the Weather_Database folder
# defining the path
output_data_path = "Resources/WeatherPy_Database.csv"

# exporting data into csv file
city_data_df.to_csv(output_data_path, index_label="city_id")